In [2]:
import pandas as pd
import numpy as np
from dplython import (DplyFrame, X, select, sift, sample_n,
                      sample_frac, head, arrange, mutate, group_by, summarize, DelayFunction)

In [3]:
combine = '/Users/kiefer/github-repos/DSI-SF-2/datasets/football_combine/combine.csv'

In [5]:
combine = DplyFrame(pd.read_csv(combine))
combine.head(2)

,id,Year,Name,Position,HeightFeet,HeightInches,Weight,Arms,Hands,FortyYD,...,Bench,Round,College,Pick,PickRound,PickTotal,FirstName,LastName,HeightInchesTotal,Wonderlic
0,8984,2013,Quanterus Smith,DE,6,5.0,250,33.25,10.375,0.0,...,0,5,Western Kentucky,13(146),13,146,Quanterus,Smith,77.0,0
1,9002,2013,Abry Jones,DT,6,3.0,313,35.00,9.750,0.0,...,30,0,NaN,NaN,0,0,Abry,Jones,75.0,0


In [6]:
# position = combine['Position']

position = (combine >> select(X.Position))
print position[0:5]

  Position
0       DE
1       DT
2       DT
3       FS
4       FS


In [7]:
# Pandas:
players = combine[['Name','Weight']]
players = players[players.Weight < 200]
players.head()

,Name,Weight
15,Brandon Boykin,182
49,Foswhitt Whittaker,193
55,Ryan Broyles,192
56,Chase Minnifield,183
58,Shaun Prater,190


In [8]:
dply_players = (combine >> 
                sift(X.Weight < 200) >>
                select(X.Name, X.Weight))
dply_players.head()

,Name,Weight
15,Brandon Boykin,182
49,Foswhitt Whittaker,193
55,Ryan Broyles,192
56,Chase Minnifield,183
58,Shaun Prater,190


In [9]:
# Take a random sample of 5% and take out the Name and Weight
sample = combine[["Name","Weight"]]
sample = sample.sample(frac=.05)
sample.head(2)

,Name,Weight
4350,Brandon Williams,335
1082,Adrian Moten,228


In [12]:
dply_sample = (combine >>
               sample_frac(0.05))
dply_sample.head(2)

,id,Year,Name,Position,HeightFeet,HeightInches,Weight,Arms,Hands,FortyYD,...,Bench,Round,College,Pick,PickRound,PickTotal,FirstName,LastName,HeightInchesTotal,Wonderlic
3302,9799,2011,Lee Smith,TE,6,6.0,266,34.375,9.75,4.94,...,25,5,Marshall,28(159),28,159,Lee,Smith,78.0,0
177,12584,2001,Chris Chambers,WR,6,1.0,210,0.000,0.00,4.33,...,0,2,Wisconsin,21(52),21,52,Chris,Chambers,73.0,0


In [16]:
# Group by Position and Year, add together weight and arms, 
# and return the mean of that column
def weight_plus_arms_mean(df):
    summed = df['Weight'] + df['Arms']
    df['weight_arms_mean'] = np.mean(summed)
    return df.loc[:, 'weight_arms_mean'].iloc[0]

combine.groupby(['Position','Year']).apply(weight_plus_arms_mean).reset_index().head(3)

,Position,Year,0
0,C,2014,307.166667
1,CB,1999,184.807692
2,CB,2000,191.750000


In [20]:
# dplyr version:
wa_mean = (combine >> 
           group_by(X.Position, X.Year) >>
           mutate(weight_arms_sum=X.Weight + X.Arms) >>
           summarize(weight_arms_mean=X.weight_arms_sum.mean(),
                     year_mean=X.Year.mean()))
wa_mean.head(3)

,Position,Year,weight_arms_mean,year_mean
0,C,2014,307.166667,2014
1,CB,1999,184.807692,1999
2,CB,2000,191.750000,2000


In [19]:
@DelayFunction
def mean_of_columns(df):
    meancol = df.mean(axis=1)
    return meancol

# mean of year, weight and hands columns:
ywh_mean = (combine >>
            select(X.Name, X['Weight'], X.Year, X.Hands) >>
            mutate(mean_of_ywh = (combine >>
                                  select(X.Year, X.Weight, X.Hands) >>
                                  mean_of_columns(X))))

ywh_mean.head()

,Name,Weight,Year,Hands,mean_of_ywh
0,Quanterus Smith,250,2013,10.375,757.791667
1,Abry Jones,313,2013,9.750,778.583333
2,Bennie Logan,309,2013,10.250,777.416667
3,John Boyett,204,2013,8.500,741.833333
4,Bacarri Rambo,211,2013,9.250,744.416667
